In [ ]:
import serial.tools.list_ports as port_list
ports = list(port_list.comports())
for port in ports:
    print(f"{port}")


In [ ]:
from time import sleep
from threading import Thread, Lock
from time import time

thread: dict = {
    "thread": None,
    "data": list(),
    "run_flag": True,
    "lock": Lock()
}


def read_serial(port: str = "COM9") -> None:
    from serial import Serial

    with Serial(port=port, baudrate=115200, timeout=1) as serial:
        # Wait for system to start
        sleep(2)
        line: str = serial.readline().decode()[0:-1]
        assert (
            line == "System ready"), f"Expected 'System ready', got '{line}'"

        # Run until kill signal
        while thread["run_flag"]:
            # Clear the read buffer
            serial.flushInput()

            # Read from the device
            line: str = serial.readline().decode()

            try:
                # Split the values in substrings
                values_str: str = line.split(",")

                # Create a dict
                values: dict = {
                    "Time": time(),
                    "A0": float(values_str[0]),
                    "A1": float(values_str[1]),
                    "A2": float(values_str[2])
                }

                with thread["lock"]:
                    # Store the data
                    thread["data"].append(values)
            except:
                pass
        print("Thread ended")


# Start the thread
thread["thread"] = Thread(target=read_serial, args=("COM9",))
thread["thread"].start()
print("Thread started")

for _ in range(10):
    # TODO SIGGI HERE
    data:list = list()
    # Get the data and clear the list
    with thread["lock"]:
        data = thread["data"].copy()
        thread["data"].clear()
    # Use the data
    print(data)
    sleep(1)

# End the thread
thread["run_flag"] = False
thread["thread"].join()
